In [2]:
# Schizophrenia classification problem

import numpy as np
import pandas as pd
import os
import mne
from glob import glob
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV
import zipapp


In [11]:
import numpy as np
import pandas as pd
import os
import mne
from glob import glob
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import GroupKFold, GridSearchCV


In [26]:
edf_files=glob('extraction_directory/*.edf')

In [27]:
print(len(edf_files))

28


In [28]:
#edf_files='k/extraction_directory/h03.edf'

In [31]:
healthy_file_path=[i for i in edf_files if 'h' in i.split('/')[1]]


In [32]:
patient_file_path=[i for i in edf_files if 's' in i.split('/')[1]]

In [33]:
# Data preprocessing
def read_Data(file_path):
    # Load the EEG data from the EDF file with preload=True
    data=mne.io.read_raw_edf(file_path,preload=True)
    # Set EEG reference (if not already set)
    data.set_eeg_reference()
    # Apply a bandpass filter from 0.5 Hz to 45 Hz
    data.filter(l_freq=0.5,h_freq=45)
    # Apply a notch filter to remove power line noise (50 Hz)
    data.notch_filter(freqs=50)
    # Create epochs with a fixed length of 5 seconds and 1-second overlap
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    # Load data into memory (necessary for ICA)                                   )
    epochs.load_data()
    # Apply Independent Component Analysis (ICA) for artifact removal                                    
    ica = mne.preprocessing.ICA(n_components=5, random_state=97, max_iter=800)
    ica.fit(epochs)
    epochs = ica.apply(epochs)
    # Reject epochs with high amplitude (adjust the threshold as needed)
    epochs.drop_bad(reject={'eeg': 100e-6})                                   
     
    #Baseline correction (adjust the baseline period as needed)
    epochs.apply_baseline((0, 4.996))  # Correct the baseline to the period from -0.5 to 0 seconds
    bad_channels = epochs.info['bads']
    # Reset bad channels (if desired)
    epochs.info['bads'] = []
     # Get the EEG data as a NumPy array  
    # Spatial Filtering (e.g., surface Laplacian)
    epochs.set_eeg_reference('average', projection=False)
        
        # Montage Selection (replace with your specific montage)
    #montage = mne.channels.read_custom_montage(data)
    #epochs.set_montage(montage)
        
        # Decimation (downsampling to a lower sampling rate)
    epochs.resample(100)  # Replace '100' with your desired new sampling rate
            
    array=epochs.get_data()
    
    
    return array
    

In [35]:
file_path=healthy_file_path[0]
sample_data=read_Data(file_path)

Extracting EDF parameters from /home/pawan/Downloads/Ghanim_machine_learning/data_code/extraction_directory/h03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop fi

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
227 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 227 events and 1250 original time points ...
0 bad epochs dropped
Fitting ICA to data using 19 channels (please be patient, this may take a while)


[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


Selecting by number: 5 components
Fitting ICA took 1.2s.
Applying ICA to Epochs instance
    Transforming to ICA space (5 components)
    Zeroing out 0 ICA components
    Projecting back using 19 PCA components
    Rejecting  epoch based on EEG : ['Fp2', 'T6', 'Fp1', 'F7', 'T5', 'O1']
    Rejecting  epoch based on EEG : ['Fp2', 'Fp1']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp2', 'T6', 'O2', 'Fp1', 'F7', 'T5', 'O1', 'P4', 'F3']
    Rejecting  epoch based on EEG : ['Fp2', 'O2', 'Fp1', 'T5', 'O1']
    Rejecting  epoch based on EEG : ['Fp2', 'Fp1']
    Rejecting  epoch based on EEG : ['Fp2', 'T6', 'O2', 'Fp1', 'F7', 'T5', 'F3']
    Rejecting  epoch based on EEG : ['Fp2', 'F8', 'T6', 'O2', 'Fp1', 'F7', 'T3', 'T5', 'O1', 'F4', 'F3']
    Rejecting  epoch based on EEG : ['Fp2', 'F8', 'Fp1', 'F7', 'T5']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp2', 'Fp1']
    Rejecting  epoch based on EEG : ['Fp2', 'Fp1', 'F7']

In [36]:
sample_data.shape

(201, 19, 500)

In [37]:
%%capture
control_epochs_array=[read_Data(i) for i in healthy_file_path]
patient_epochs_array=[read_Data(i) for i in patient_file_path]

In [38]:
# Creating labels
control_epoch_labels=[len(i)*[0] for i in control_epochs_array]
patient_epoch_labels=[len(i)*[1] for i in patient_epochs_array]
len(control_epoch_labels),len(patient_epoch_labels)

(14, 14)

In [39]:
data_list=control_epochs_array+patient_epochs_array
label_list=control_epoch_labels+patient_epoch_labels

In [40]:
# Create a group list
group_list=[[i]*len(j) for i,j in enumerate(data_list)]

In [41]:
len(group_list)

28

In [43]:
# Convert the list to array
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(group_list)
print(data_array.shape,label_array.shape,group_array.shape)


(5482, 19, 500) (5482,) (5482,)


In [46]:
#Now
# Extract feature from the raw data
from scipy import stats
def mean(x):
    return np.mean(x,axis=-1)
def std(x):
    return np.std(x, axis=-1)
def ptp(x):
    return np.ptp(x, axis=-1)
def var(x):
    return np.var(x,axis=-1)
def minim(x):
    return np.min(x,axis=-1)
def maxim(x):
    return np.max(x,axis=-1)
def argminim(x):
    return np.argmin(x,axis=-1)
def argmaxim(x):
    return np.argmax(x,axis=-1)
def sqrt(x):
    return np.sqrt(np.mean(x ** 2, axis=-1))
def abs_diff_sigma(x):
    return np.sum(np.abs(np.diff(x,axis=-1)),axis=-1)
def skew(x):
    return stats.skew(x,axis=-1)
def kurtosis(x):
    return stats.kurtosis(x,axis=-1)
def concatenate_features(x):
    return np.concatenate((mean(x),std(x),ptp(x),var(x),minim(x),maxim(x),argminim(x),
 argmaxim(x),sqrt(x),abs_diff_sigma(x),skew(x),
    kurtosis(x)),axis=-1)



    

In [47]:
features=[]
for d in data_array:
    features.append(concatenate_features(d))

In [48]:
features_array=np.array(features)
features_array.shape

(5482, 228)

In [49]:
#Model
clf=LogisticRegression(max_iter=1000)
gkf=GroupKFold(5)
pipe=Pipeline([('scaler',StandardScaler()),('clf',clf)])
param_grid={'clf__C':[.1,.5,.7,1,3,5,7]}
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=12)
gscv.fit(features_array,label_array,groups=group_array)


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf',
                                        LogisticRegression(max_iter=1000))]),
             n_jobs=12, param_grid={'clf__C': [0.1, 0.5, 0.7, 1, 3, 5, 7]})

In [50]:
gscv.best_score_

0.7265968617950622